In [71]:
import joblib
import pickle

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import json

In [64]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# input_dim = X.shape[1]  # Jumlah fitur
input_dim = 36
hidden_dim = 64  # Ukuran layer tersembunyi
output_dim = 1  # Output tunggal (num_of_harvest)

In [140]:
values_dict = {
 'average_weight': 6.158,
 'evening_do': 5.64,
 'evening_temperature': 30.0,
 'fasting': 1,
 'last_weight': 6.6,
 'long_cycle': 62,
 'morning_do': 4.97,
 'morning_pH': 7.9,
 'morning_salinity': 20.68,
 'morning_temperature': 30.0,
 'size': 177,
 'start_weight': 402,
 'transparency': 38.0
}

In [141]:
with open('./feature_store/mi_features.json') as file:
    mi_feateres = json.load(file)

In [142]:
input_mi = dict()

In [143]:

# Fungsi untuk evaluasi ekspresi dari string
def evaluate_expression(expr, values):
    # Pisahkan elemen-elemen dalam key
    elements = expr.split()
    # Jika key mengandung '^2'
    if '^2' in expr:
        base = expr.replace('^2', '')
        return values[base] * values[base]
    # Jika key mengandung perkalian dua elemen
    elif len(elements) == 2:
        return values[elements[0]] * values[elements[1]]
    # Jika key adalah elemen tunggal
    else:
        return values[expr]

# Buat dictionary baru dengan key dari list dan value hasil evaluasi ekspresi
result_dict = {key: evaluate_expression(key, values_dict) for key in mi_feateres}

# Tampilkan hasil
print(result_dict)


{'last_weight': 6.6, 'average_weight': 6.158, 'size': 177, 'start_weight last_weight': 2653.2, 'start_weight average_weight': 2475.516, 'last_weight^2': 43.559999999999995, 'last_weight average_weight': 40.6428, 'last_weight fasting': 6.6, 'last_weight morning_temperature': 198.0, 'last_weight evening_temperature': 198.0, 'last_weight morning_do': 32.802, 'last_weight evening_do': 37.224, 'last_weight morning_salinity': 136.488, 'last_weight morning_pH': 52.14, 'last_weight transparency': 250.79999999999998, 'last_weight long_cycle': 409.2, 'average_weight^2': 37.920964000000005, 'average_weight fasting': 6.158, 'average_weight morning_temperature': 184.74, 'average_weight evening_temperature': 184.74, 'average_weight morning_do': 30.60526, 'average_weight evening_do': 34.73112, 'average_weight morning_salinity': 127.34744, 'average_weight morning_pH': 48.6482, 'average_weight transparency': 234.00400000000002, 'average_weight long_cycle': 381.79600000000005, 'fasting size': 177, 'morn

In [144]:
result_dict.values()

dict_values([6.6, 6.158, 177, 2653.2, 2475.516, 43.559999999999995, 40.6428, 6.6, 198.0, 198.0, 32.802, 37.224, 136.488, 52.14, 250.79999999999998, 409.2, 37.920964000000005, 6.158, 184.74, 184.74, 30.60526, 34.73112, 127.34744, 48.6482, 234.00400000000002, 381.79600000000005, 177, 5310.0, 5310.0, 879.6899999999999, 998.28, 3660.36, 1398.3, 6726.0, 10974, 31329])

In [150]:
# Fungsi untuk normalisasi Min-Max
def min_max_normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

# Lakukan normalisasi pada setiap item dalam data_to_normalize
normalized_data = {}
for key, value in result_dict.items():
    min_value = min_max_json["min"][key]
    max_value = min_max_json["max"][key]
    normalized_data[key] = min_max_normalize(value, min_value, max_value)

normalized_data

{'last_weight': 0.07742998352553541,
 'average_weight': 0.07222875970816664,
 'size': 0.12391690687106707,
 'start_weight last_weight': 0.36722487846227697,
 'start_weight average_weight': 0.34263193592425945,
 'last_weight^2': 0.006029010714408897,
 'last_weight average_weight': 0.0056252459861727885,
 'last_weight fasting': 0.1388304585612116,
 'last_weight morning_temperature': 0.08152885437343341,
 'last_weight evening_temperature': 0.0775077361872742,
 'last_weight morning_do': 0.07742998352553541,
 'last_weight evening_do': 0.07742650631108927,
 'last_weight morning_salinity': 0.06959591515139112,
 'last_weight morning_pH': 0.07610039617551391,
 'last_weight transparency': 0.07742998352553543,
 'last_weight long_cycle': 0.043725841264136936,
 'average_weight^2': 0.005248521259433704,
 'average_weight fasting': 0.20323432343234324,
 'average_weight morning_temperature': 0.07605340558396964,
 'average_weight evening_temperature': 0.07230128947074302,
 'average_weight morning_do': 0

In [130]:
with open('./min_max_values.json', 'r') as file:
    min_max_json = json.load(file)

In [131]:
result_dict_norm = normalize_dict(result_dict, './min_max_values.json')

In [145]:
result_dict_norm  = {
  "last_weight": 0.0774,
  "average_weight": 0.0722,
  "size": 0.1239,
  "start_weight last_weight": 0.3672,
  "start_weight average_weight": 0.3426,
  "last_weight^2": 0.0060,
  "last_weight average_weight": 0.0056,
  "last_weight fasting": 0.1388,
  "last_weight morning_temperature": 0.0815,
  "last_weight evening_temperature": 0.0775,
  "last_weight morning_do": 0.0774,
  "last_weight evening_do": 0.0774,
  "last_weight morning_salinity": 0.0696,
  "last_weight morning_pH": 0.0761,
  "last_weight transparency": 0.0774,
  "last_weight long_cycle": 0.0437,
  "average_weight^2": 0.0052,
  "average_weight fasting": 0.2032,
  "average_weight morning_temperature": 0.0761,
  "average_weight evening_temperature": 0.0723,
  "average_weight morning_do": 0.0722,
  "average_weight evening_do": 0.0722,
  "average_weight morning_salinity": 0.0649,
  "average_weight morning_pH": 0.0710,
  "average_weight transparency": 0.0722,
  "average_weight long_cycle": 0.0408,
  "fasting size": 0.1308,
  "morning_temperature size": 0.1333,
  "evening_temperature size": 0.1238,
  "morning_do size": 0.1244,
  "evening_do size": 0.1236,
  "morning_salinity size": 0.1102,
  "morning_pH size": 0.1239,
  "transparency size": 0.1239,
  "long_cycle size": 0.0649,
  "size^2": 0.0171
}

In [146]:
# Muat model menggunakan pickle
with open('base_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [147]:
inputs = list(result_dict_norm.values())

In [148]:
with torch.no_grad():
    prediction_normalized = loaded_model(torch.tensor(inputs))
    y_pred = prediction_normalized.numpy()

In [149]:
y_pred

array([0.22942989], dtype=float32)